# **🤗 Transformer를 사용한 다국어 ASR용 XLS-R 미세 조정**

***New (11/11)***: *이 블로그 게시물은 [XLS-R](https://huggingface.co/models?other=xls_r)*)라는 XLSR의 후속 제품으로 업데이트되었습니다.

## Notebook Setting

이 노트에서는 XLS-R[**Wav2Vec2-XLS-R-300M***](https://huggingface.co/facebook/wav2vec2-xls-r-300m) - ASR을 미세 조정할 수 있는 방법에 대해 설명한다.

XLS-R은 ASR 및 필기 인식과 같은 시퀀스 간 문제에 대한 신경망을 훈련하는 데 사용되는 알고리듬인 연결주의 시간 분류(CTC)를 사용하여 미세 조정된다.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 18 19:05:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

datasets과 transformers, 오디오 파일을 로드하는 'torchaudio'와 [WER](https://huggingface.co/metrics/wer) metrics를 사용하여 미세 조정 모델을 평가하는 'jiwer'를 설치

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

학습 중에는 교육 체크포인트를 허깅페이스에 직접 업로드하는 것이 좋다.

그러려면 허그 페이스 웹사이트에서 인증 토큰을 저장해야 합니다. 아직 가입하지 않은 경우 https://huggingface.co/join 들어가서 가입해야함

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


그런 다음 Git-LFS를 설치하여 모델 model checkpoints를 업로드해야 합니다.

In [ ]:
%%capture
!apt install git-lfs

In [ ]:
!git lfs install

## Prepare Data, Tokenizer, Feature Extractor

**Create `Wav2Vec2CTCTokenizer`**

**zeroth 데이터 셋은 Kaldi open source tool-kit을 사용해서 한국어 음성인식기를 구현하는 프로젝트에서 사용된 데이터로 발화시간 : 51.6시간 (Train 22,263발화 & Test 457발화)으로 구성되어 있다.**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("kresnik/zeroth_korean")

Downloading:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset zeroth_korean_asr downloaded and prepared to /root/.cache/huggingface/datasets/kresnik___zeroth_korean_asr/clean/1.0.1/f6cf96a53d5512525e3113bab8048d36ce268658d6e0c40d45f65dfa3f0bc343. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
common_voice_train = dataset['train']
common_voice_test = dataset['test']

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 457
    })
})

### 많은 데이터 세트는 각 오디오 배열 'audio'에 대한 'text'과 파일 'path'만을 제공함. 우리는 미세 조정을 위해 전사된 텍스트만 고려해 가져다 씀.

In [ ]:
common_voice_train = common_voice_train.remove_columns(["speaker_id", "chapter_id", "id"])
common_voice_test = common_voice_test.remove_columns(["speaker_id", "chapter_id", "id"])

데이터 세트의 일부 랜덤 샘플을 표시하는 짧은 함수를 작성합니다. 몇 번을 반복해서 기록을 확인해 봐요.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["file", "audio"]), num_examples=10)

,text
0,한편 대설주의보가 발효 중인 전남 해안 지역에서는 약하게 눈이 이어지고 있습니다
1,부동산 백 십 사 는 수도권 아파트 매매가격이 구 월 한달간 평균 영 쩜 칠 육 퍼센트 상승했다고 이 일 밝혔다
2,피부질환을 전문으로 치료하는 생기한의원 서초점이 국내 의료계 최초로 피부질환 치료에 미세현미경을 이용한 미세피부치료를 도입해 화제가 되고 있다
3,삼성그룹의 시작점으로 상징성이 큰 삼성본관도 지금 당장은 팔 계획이 없는 것으로 파악됐지만 장기적으로는 매각 가능성이 있는 자산으로 꼽힌다
4,정연국 대변인은 국민이 큰 충격에 빠져 있는데 이른 시일 내에 귀국해 의혹을 해소시켜야 한다고 촉구했다
5,매일 고픈 배를 움켜쥔 채 잠드는 사람이 팔 억 명이나 되는데 몇 명이서 그런 막대한 부를 틀어쥐고 있다는 것은 도저히 납득할 수 없는 일이다
6,경찰은 회사 관계자와 유가족 등을 대상으로 정확한 투신 경위에 대해 파악할 예정이다
7,재선의 정성호 의원도 책임정치 실현을 위해 물러난 전임 지도부들은 의무를 다한 것이 아니고 사심 때문이었는가라며 이중적 잣대이자 견강부회라고 지적했다
8,경제성장률 마이너스 이 쩜 팔 퍼센트 이 쩜 육 퍼센트 오바마 대통령의 취임 첫해 마이너스 이 쩜 팔 퍼센트였던 경제 성장률은 이천 십 년 이 쩜 오 퍼센트로 반전에 성공했다
9,다른 동물의 생체 주기 즉 생체 리듬도 박테리아의 영향을 받을 수 있기 때문입니다


데이터 전처리

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_remove_regex, '', batch["text"]).lower()
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

처리된 text labels을 다시 살펴봅시다.

In [ ]:
show_random_elements(common_voice_train.remove_columns(["file", "audio"]), num_examples=10)

,text
0,여야 불문 유력 차기 대권주자들이 김 전 대통령의 정치적 후계자 자리를 놓고 경쟁하는 모양새가 이어지고 있다
1,딸은 병원에서 한 달가량 치료받다가 지난 일 월 이십 칠 일 뇌 손상으로 숨졌다
2,국정원 직원이 일반 네티즌인 것처럼 아름다운 단일화는 말도 안 된다는 글들이었습니다
3,하지만 왕 회장은 로이드리스트 인터뷰에서 상황이 어떻든 용선료 인하를 받아들일 수 없다고 강조했다
4,임씨가 죽으면서 내국인 사찰과 선거용 사찰은 없었다고 밝혔지만 오히려 그 의혹은 증폭되고 있는 것이다
5,이재용 부회장 등은 무려 육 조 원의 세금을 어떻게 마련할까
6,단기간 너무 많이 오른 탓인지 거래량이 빠르게 줄고 시세 상승세도 주춤하다
7,북한이 대북 제재 결의가 하루도 지나지 않아 말이 아닌 행동으로 반격에 나선 것은 매우 이례적이다
8,새로 주택을 짓는 택지공급을 축소하고 대출 심사를 강화하겠다는 내용이었습니다
9,안 의원의 선진화법 개정 찬성 주장으로 현재 정국 이슈로 부상한 선진화법 논의에 변화가 뒤따를지 주목된다


CTC에서는 음성 덩어리를 문자로 분류하는 것이 일반적이므로 여기서도 동일하게 수행합니다.
훈련 및 테스트 데이터의 모든 구별되는 문자를 추출하고 이 문자 집합에서 어휘를 구축해 봅시다.

우리는 모든 전사를 하나의 긴 전사로 연결한 다음 문자열을 문자 집합으로 변환하는 매핑 함수를 작성한다.
'batched ='라는 주장을 전달하는 것이 중요하다.true는 'map(...)' 함수에 적용되므로 매핑 함수는 모든 문자 변환에 한 번에 액세스할 수 있습니다.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

이제, 우리는 훈련 데이터 세트와 테스트 데이터 세트에서 모든 별개의 문자의 조합을 만들고 결과 목록을 열거된 사전으로 변환한다.

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
len(vocab_dict)

1203

"""가 고유한 토큰 클래스를 가지고 있다는 것을 명확히 하기 위해, 우리는 그것에게 좀 더 눈에 띄는 문자 "|caint"를 제공한다. 또한 "알 수 없는" 토큰을 추가하여 모델이 나중에 zeroth 데이터 셋에서 마주치지 않는 캐릭터를 처리할 수 있도록 한다.

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

마지막으로 CTC의 "*공백 토큰*"에 해당하는 패딩 토큰도 추가한다. 빈 토큰은 CTC 알고리즘의 핵심 구성 요소이다. 자세한 내용은 "Alignment" 섹션 [여기](https://distill.pub/2017/ctc/)을 참조하십시오.

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

1205


우리의 어휘는 1205개의 토큰으로 구성되어 있습니다. 이것은 우리가 사전 훈련된 XLS-R 체크포인트 위에 추가할 선형 레이어의 출력 치수가 1205라는 것을 의미함.

어휘를 json 파일로 저장해 보겠습니다.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

마지막 단계에서는 json 파일을 사용하여 "Wav2Vec2CTokenizer" 클래스의 인스턴스에 어휘를 로드한다.

In [ ]:
from transformers import Wav2Vec2CTCTokenizer 

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='./', vocab_size=1422, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]})

미세조정 모델로 방금 만든 토큰화를 다시 사용하려면 토큰화를 [🤗허브](https://huggingface.co/))에 업로드하는 것이 좋다. 파일을 업로드할 리포를 호출함.
`"wav2vec2-large-xls-r-300m-korean-colab"`:

In [ ]:
repo_name ="wav2vec2-large-xls-r-300m-korean-colab"

토큰화를 [🤗 Hub](https://huggingface.co/)에 업로드합니다.

In [ ]:
tokenizer.push_to_hub(repo_name)

좋습니다. 방금 생성된 저장소는 `https://huggingface.co/<your-username>/wav2vec2-large-xls-r-300m-korean-colab`에서 확인할 수 있습니다.

### Create `Wav2Vec2FeatureExtractor`

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

사용자 친화성을 향상시키기 위해 기능 추출기와 토큰화기는 단일 'Wav2Vec2Processor' 클래스로 *랩*되어 '모델'과 '프로세서' 개체만 필요하다.

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

다음으로, 우리는 데이터 세트를 준비할 수 있다.

### Preprocess Data

'path'는 오디오 파일의 절대 경로를 나타냄


In [ ]:
common_voice_train[0]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/217/217_003_0002.flac'

XLS-R은 16kHz의 1차원 배열 형식의 입력을 예상합니다. 즉, 오디오 파일을 로드하고 다시 샘플링해야 합니다.

고맙게도 '데이터 세트'는 다른 열을 '오디오'로 호출하여 자동으로 이 작업을 수행합니다.

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/217/217_003_0002.flac',
 'array': array([-6.1035156e-05, -1.5258789e-04, -1.2207031e-04, ...,
         4.6997070e-03,  4.2419434e-03,  4.2114258e-03], dtype=float32),
 'sampling_rate': 16000}

In [ ]:
common_voice_train[0]["text"]

'이 사고로 크게 다친 서씨가 인근 병원으로 옮겨졌으나 의식이 없는 상태다'

오디오 파일을 즉시 로드하고 다시 샘플링하는 새로운 오디오 기능 덕이다.
위의 예에서 오디오 데이터가 48kHz의 샘플링 속도로 로드되는 반면 모델은 16kHz를 예상합니다

In [ ]:
from datasets.features import Audio
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

오디오를 다시 보자.

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/217/217_003_0002.flac',
 'array': array([-6.1035156e-05, -1.5258789e-04, -1.2207031e-04, ...,
         4.6997070e-03,  4.2419434e-03,  4.2114258e-03], dtype=float32),
 'sampling_rate': 16000}

데이터 세트를 더 잘 이해하고 오디오가 올바르게 로드되었는지 확인하기 위해 오디오 파일 몇 개를 들어 보겠습니다.

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["text"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

올해 들어 한국 경제를 위협하는 가장 큰 요인으로 지목됐던 미국 중국 등 주요 두 개국 리스크 가운데 먼저 중국의 경기 둔화가 충격을 줬다


이제 데이터가 올바르게 로드되고 다시 샘플링된 것 같습니다.

말하는 사람이 말하는 속도, 억양, 배경 환경 등에 따라 변하는 것을 들을 수 있다. 전반적으로, 녹음은 허용할 만큼 명확하게 들리지만, 이는 크라우드 소싱 읽기 음성 코퍼스에서 예상된다.

음성 입력의 모양, 전사 및 해당 샘플링 속도를 인쇄하여 데이터가 올바르게 준비되었는지 최종 확인해 보겠습니다.

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["text"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: 기자 평화와 안정을 단번에 깨뜨릴 수 있는 북한 핵은 중국 입장에서 보더라도 반드시 막아야 되는 일임에는 틀림이 없습니다
Input array shape: (152588,)
Sampling rate: 16000


데이터는 1차원 배열이고 샘플링 속도는 항상 16kHz에 해당하며 대상 텍스트는 정규화됩니다.

마지막으로, 우리는 'Wav2Vec2Processor'를 활용하여 'Wav2Vec2For'가 예상하는 형식으로 데이터를 처리할 수 있다.훈련을 위한 CTC'. 그러기 위해 Dataset의 map 함수를 사용해 봅시다.

첫째, 단순히 'batch["audio"]를 호출함으로써 오디오 데이터를 로드하고 다시 샘플링한다.

둘째, 우리는 로드된 오디오 파일에서 'input_values'를 추출한다. 이 경우 Wav2Vec2 프로세서는 데이터를 정규화할 뿐입니다.

셋째, 우리는 전사를 레이블 ID로 인코딩한다.

**참고**: 이 매핑 함수는 'Wav2Vec2Processor' 클래스를 사용하는 방법을 보여주는 좋은 예입니다. "정상" 컨텍스트에서 'processor(...)'를 호출하면 'Wav2Vec2Feature'로 리디렉션됩니다.추출기의 호출 메서드입니다. 그러나 프로세서를 'as_target_processor' 컨텍스트로 래핑할 때 동일한 메서드가 'Wav2Vec2CTokenizer'의 호출 메서드로 리디렉션됩니다.

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

데이터의 모든 예에 map을 적용해 봅시다.

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

재는 오디오 로딩과 리샘플링을 위해 torchaudio 와 librosa를 사용하고 있다. 독자적인 맞춤형 데이터 로딩/샘플링을 구현하고 싶다면 '경로' 열을 사용하고 '오디오' 열은 무시해도 된다.

긴 입력 시퀀스는 많은 메모리를 필요로 한다. XLS-R은 '셀프 어텐션'을 기반으로 메모리 요구 사항은 긴 입력 시퀀스에 대한 입력 길이에 따라 2차적으로 확장된다.
가지고 있는 자원의 상황에 맞게 음성길이를 짤라서 사용하면 된다. 

In [ ]:
# max_input_length_in_sec = 3.0 -> num_rows: 0
# max_input_length_in_sec = 4.0 -> num_rows: 54
# max_input_length_in_sec = 5.0 -> num_rows: 1230
# max_input_length_in_sec = 6.0 -> num_rows: 4313
# max_input_length_in_sec = 7.0 -> num_rows: 7978
# max_input_length_in_sec = 8.0 -> num_rows: 11385
# max_input_length_in_sec = 8.5 -> num_rows: 13004
# max_input_length_in_sec = 9.0 -> num_rows: 14437
# max_input_length_in_sec = 10.0 -> num_rows: 16829

In [ ]:
max_input_length_in_sec = 8.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/23 [00:00<?, ?ba/s]

In [ ]:
common_voice_train

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 10877
})

## Training

- 데이터 수집기를 정의합니다. 대부분의 NLP 모델과 달리 XLS-R은 출력 길이보다 입력 길이가 훨씬 큽니다. *예: *입력 길이 50000의 샘플은 출력 길이가 100 이하입니다. 입력 크기가 크다는 점을 고려할 때, 훈련 배치를 동적으로 패딩하는 것이 훨씬 더 효율적이다. 즉, 모든 훈련 샘플은 전체적으로 가장 긴 샘플이 아니라 배치에서 가장 긴 샘플로만 패딩되어야 한다. 따라서 XLS-R을 미세 조정하려면 특수 패딩 데이터 수집기가 필요하며, 이를 아래에 정의합니다.

- 평가 메트릭입니다. 훈련 중에, 모델은 단어 오류율로 평가되어야 한다. 우리는 그에 따라 "compute_metrics" 함수를 정의해야 한다.

- 사전 교육된 체크포인트를 로드합니다. 우리는 미리 훈련된 체크포인트를 로드하고 훈련을 위해 정확하게 구성해야 한다.

- 교육 구성을 정의합니다.

모델을 미세 조정한 후 테스트 데이터에서 올바르게 평가하고 음성을 올바르게 전사하는 법을 실제로 배웠는지 확인한다.

### Set-up Trainer

먼저 데이터 수집기를 정의합니다. 

일반적인 데이터 수집기와 달리 이 데이터 수집기는 너무 많은 세부 사항을 다루지 않고도 'input_values'와 'label'을 다르게 취급하므로 이들에 대한 별도의 패딩 기능에 적용된다(다시 한번 XLS-R 프로세서의 컨텍스트 관리자를 사용한다). 음성 입력과 출력의 양식이 다르기 때문에 이는 동일한 패딩 함수에 의해 처리되지 않아야 한다는 것을 의미하기 때문에 필요하다.
일반적인 데이터 수집기와 유사하게, 라벨에 "-100"이 있는 패딩 토큰은 손실을 계산할 때 ** 고려되지 않는다.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
              
    수신된 입력을 동적으로 패딩하는 데이터 수집기입니다.
    인수:
      프로세서(: 클래스:"~요리사들.Wav2Vec2 프로세서 ')
      데이터를 처리하는 데 사용되는 프로세서입니다.
      패딩(:obj:'bool', :obj:'str' 또는 :class:"~요리사들.토큰화_base_base.'선택 사항'인 패딩 전략은 기본적으로 :obj:'참':
      반환된 시퀀스를 패딩할 전략을 선택합니다(모델의 패딩 면 및 패딩 인덱스에 따라).
      다음 중:
        * :obj:True 또는 :obj:"'longest': 배치에서 가장 긴 시퀀스로 패드(또는 단일만 있는 경우 패딩 없음)
        sequence(제공된 경우).
        * :obj:'max_length': 인수:obj:'max_length'로 지정된 최대 길이 또는
        인수가 제공되지 않은 경우 모델에 대해 허용 가능한 최대 입력 길이입니다.
        * :obj:'False' 또는 :obj:'do_not_pad'(기본값): 패딩 없음(즉, 다음과 같은 시퀀스로 배치를 출력할 수 있음)
        길이가 다릅니다).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

앞서 언급했듯이, ASR의 주요 메트릭은 WER(Word Error Rate)이므로 이 코드에서 적용할 것 입니다.

In [ ]:
from datasets import load_metric

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

이 모델은 로짓 벡터의 시퀀스를 반환 : 
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

로짓 벡터 $\mathbf{y}_1$는 우리가 이전에 정의한 어휘에서 각 단어에 대한 로그 데이터 세트를 포함하므로 $\text{len}(\mathbf{y}_i) =$ 'config.vocab_size'이다. 우리는 모델의 가장 가능성 있는 예측에 관심이 있으므로 로짓의 "argmax(...)"를 취한다. 또한, 우리는 '-100'을 'pad_token_id'로 대체하고 ID를 디코딩하여 인코딩된 레이블을 원래 문자열로 다시 변환하는 동시에 연속 토큰이 CTC 스타일 ${}^1$에서 동일한 토큰으로 **not* 그룹화되도록 한다.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

**Note**: 이 코드를 사용하여 다른 언어로 XLS-R을 교육하는 경우 이러한 하이퍼 파라미터 설정이 제대로 작동하지 않을 수 있습니다. 사용 사례에 따라 자유롭게 조정할 수 있습니다.

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    # "teddy322/wav2vec2-large-xls-r-300m-korean",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Downloading:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

위와같이 학습을 시키다가 허깅페이스로 push했을때 다시 그 repo를 불러와서 학습 진행이 가능하다.

XLS-R의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미 있지만 문맥적으로 독립적인 특징을 추출하는 데 사용되는 CNN 레이어 스택으로 구성된다. 모델의 이 부분은 사전 훈련 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이은 더 이상 미세 조정할 필요가 없다.
따라서 feature_extractor 부분의 모든 매개변수에 대해 'require_grad'를 'False'로 설정할 수 있다.

In [ ]:
model.freeze_feature_extractor()

마지막 단계에서, 우리는 훈련과 관련된 모든 매개 변수를 정의한다.
일부 매개 변수에 대한 자세한 설명은 다음을 참조하십시오.
- 'group_by_length'는 입력 길이가 유사한 훈련 샘플을 하나의 배치로 그룹화하여 훈련을 더 효율적으로 만든다. 이는 모델을 통해 전달되는 불필요한 패딩 토큰의 전체 수를 크게 줄임으로써 훈련 시간을 크게 단축할 수 있다.
- "learning_rate"와 "weight_decay"는 미세 조정이 안정될 때까지 휴리스틱하게 조정되었다. 이러한 매개 변수는 공통 음성 데이터 세트에 크게 의존하며 다른 음성 데이터 세트에는 최적이 아닐 수 있습니다.

다른 매개 변수에 대한 자세한 내용은 [문서](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments))를 참조하십시오.

훈련 중 체크포인트는 400개의 훈련 단계마다 허브에 비동기적으로 업로드됩니다. 또한 모델이 아직 교육 중인 동안에도 데모 위젯을 사용할 수 있습니다.

**참고**: 모델 체크포인트를 허브에 업로드하지 않으려면 'push_to_hub=False'를 설정하면 됩니다.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=20,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

이제 모든 사례를 강사에게 전달할 수 있으며 교육을 시작할 준비가 되었습니다!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/teddy322/wav2vec2-large-xls-r-300m-kor-11385-4 into local empty directory.
Using amp fp16 backend


${}^1$ 모델이 스피커 속도와 독립적이 될 수 있도록 CTC에서 동일한 연속 토큰은 단순히 단일 토큰으로 그룹화된다. 그러나 인코딩된 레이블은 모델의 예측 토큰과 일치하지 않으므로 디코딩할 때 그룹화되지 않아야 하며, 이는 'group_tokens=False' 매개 변수를 전달해야 하는 이유이다. 이 매개 변수를 전달하지 않으면 "hello"와 같은 단어가 잘못 인코딩되어 "helo"로 디코딩됩니다.

${}^2$ 빈 토큰은 모델이 두 l 사이에 빈 토큰을 삽입하도록 강제하여 "안녕"과 같은 단어를 예측할 수 있게 한다. 우리 모델의 "hello"에 대한 CTC 적합 예측은 "[PAD] [PAD] "h" "e" "l" [PAD] "l" "o" [PAD]이다.

### Training(학습)

교육은 이 노트북에 할당된 GPU에 따라 여러 시간이 소요됩니다. 훈련된 모델은 *Common Voice*의 터키어 테스트 데이터에 대해 다소 만족스러운 결과를 산출하지만, 최적으로 미세 조정된 모델은 결코 아니다. 이 노트북의 목적은 ASR 데이터 세트에서 XLS-R을 미세 조정하는 방법을 보여주는 것이다.

이 google colab을 사용하여 모델을 미세 조정하려는 경우, 비활성화로 인해 교육이 중단되지 않도록 해야 합니다. 이를 방지하기 위한 간단한 해킹은 다음 코드를 이 탭의 콘솔에 붙여넣는 것입니다(*오른쪽 마우스 클릭 -> 검사 -> 콘솔 탭 및 코드 삽입*).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

구글 콜랩에 할당된 GPU에 따라 여기서 "out-of-memory" 오류가 발생할 수 있습니다. 이 경우 `per_device_train_batch_size`를 8 이하로 줄이고 [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments)을 늘리는 것이 가장 좋을 것입니다.

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 10877
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 3400
/usr/local/lib/python3.8/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like 

Step,Training Loss,Validation Loss,Wer
400,0.027000,0.242128,0.142900
800,0.044600,0.277086,0.169402
1200,0.035600,0.260522,0.152838
1600,0.027800,0.264812,0.148472
2000,0.023000,0.254477,0.137780
2400,0.018900,0.254344,0.135070


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 457
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xls-r-300m-kor-11385-4/checkpoint-400
Configuration saved in wav2vec2-large-xls-r-300m-kor-11385-4/checkpoint-400/config.json
Model weights saved in wav2vec2-large-xls-r-300m-kor-11385-4/checkpoint-400/pytorch_model.bin
Configuration saved in wav2vec2-large-xls-r-300m-kor-11385-4/checkpoint-400/preprocessor_config.json
Configuration saved in wav2vec2-large-xls-r-300m-kor-11385-4/preprocessor_config.json
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the cu

training loss , validation , WER가 점점 내려가고있으면 잘 학습되고 있는것입니다.

결과값을 허깅페이스에 업로드 합니다.

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xls-r-300m-kor-11385-3
Configuration saved in wav2vec2-large-xls-r-300m-kor-11385-3/config.json
Model weights saved in wav2vec2-large-xls-r-300m-kor-11385-3/pytorch_model.bin
Configuration saved in wav2vec2-large-xls-r-300m-kor-11385-3/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/1.18G [00:00<?, ?B/s]

Upload file runs/Dec12_02-18-24_428bb28eec11/events.out.tfevents.1670812206.428bb28eec11.86.0:  40%|###9      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/teddy322/wav2vec2-large-xls-r-300m-kor-11385-3
   9923693..bb7368c  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/teddy322/wav2vec2-large-xls-r-300m-kor-11385-3
   9923693..bb7368c  main -> main

Dropping the following result as it does not have all the necessary field:
{'dataset': {'name': 'zeroth_korean_asr', 'type': 'zeroth_korean_asr', 'args': 'clean'}}
To https://huggingface.co/teddy322/wav2vec2-large-xls-r-300m-kor-11385-3
   bb7368c..ac1a8d5  main -> main

   bb7368c..ac1a8d5  main -> main



'https://huggingface.co/teddy322/wav2vec2-large-xls-r-300m-kor-11385-3/commit/bb7368c8bed32f70b50a8f0dc7e757eb2b3f3e80'

이제 이 모델을 모든 친구, 가족들과 공유할 수 있습니다.
예를 들어 다음과 같은 "사용자 이름/사용자가 선택한 이름"이라는 식별자와 함께 로드할 수 있습니다.

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
```

### Evaluation

최종 점검으로 모델을 로드하고 실제로 한국어 음성을 잘 학습 했는지를 확인해 봅시다.

먼저 사전 훈련된 체크포인트를 로드합니다.

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("teddy322/wav2vec2-large-xls-r-300m-kor-11385").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("teddy322/wav2vec2-large-xls-r-300m-kor-11385")

이제 테스트 세트의 첫 번째 예를 들어 모델을 실행하고 로짓의 "argmax(...)"를 가져와서 예측된 토큰 ID를 검색할 것이다.

In [ ]:
input_dict = processor(common_voice_test[1]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

In [ ]:
common_voice_test

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 457
})

In [ ]:
common_voice_test[0]

우리는 데이터 세트 인스턴스가 더 이상 원래 문장 레이블을 포함하지 않도록 'common_voice_test'를 꽤 많이 적용했다. 따라서 원본 데이터 세트를 재사용하여 첫 번째 예제의 레이블을 얻는다.

In [ ]:
common_voice_test_transcription = dataset['test']

마지막으로 예제를 디코딩할 수 있습니다.

In [ ]:
common_voice_test_transcription

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 457
})

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["text"].lower())

Prediction:
분명히 배당 률이크게 변할 때는 무슨 일이 일어나고 있다는 얘기였다고 밝혔다

Reference:
이 일 자동차업계에 따르면 임팔라는 지난달 삼십 일 일 하루에만 구백 대가 넘는 계약이 이뤄진 것으로 파악됐다


아직 완벽하지는 않다. 모델을 조금 더 훈련시키고, 데이터 전처리에 더 많은 시간을 소비하며, 특히 디코딩을 위한 언어 모델을 사용하면 모델의 전반적인 성능을 확실히 향상시킬 수 있다.

## zeroth 데이터셋의 test의 모든 예측값을 보고싶다면 사용해보시오.

In [ ]:
preds=[]
for i in range(456):   
    input_dict = processor(common_voice_test[i]["input_values"], return_tensors="pt", padding=True)
    logits = model(input_dict.input_values.to("cuda")).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]
    pred = processor.decode(pred_ids)
    preds.append(pred)

In [ ]:
for i in range(456):
    print("Prediction:")
    print(preds[i])
    print("Reference:")
    print(common_voice_test_transcription[i]["text"].lower())
    print("\n")

# **다른 실제 음성 데이터를 Inference할때 사용**

In [ ]:
!pip install transformers

In [ ]:
import librosa
import torch
import IPython.display as display
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, AutoModelForCTC, Wav2Vec2Processor
import numpy as np
from transformers import AutoTokenizer, AutoModel

Inference 할 때, 사용하는 pretrained model를 불러오자

In [ ]:
# tokenizer = Wav2Vec2Tokenizer.from_pretrained("teddy322/hyuk_Second_SON")
# model = AutoModelForCTC.from_pretrained("teddy322/hyuk_Second_SON")
# processor = Wav2Vec2Processor.from_pretrained("teddy322/hyuk_Second_SON")
# tokenizer = Wav2Vec2Tokenizer.from_pretrained("teddy322/wav2vec2-large-xls-r-300m-kor-11385-3")
# model = AutoModelForCTC.from_pretrained("teddy322/wav2vec2-large-xls-r-300m-kor-11385-3")
# processor = Wav2Vec2Processor.from_pretrained("teddy322/wav2vec2-large-xls-r-300m-kor-11385-3")
# tokenizer = Wav2Vec2Tokenizer.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
# model = AutoModelForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
# processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
model = AutoModelForCTC.from_pretrained("42MARU/ko-spelling-wav2vec2-conformer-del-1s")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("42MARU/ko-spelling-wav2vec2-conformer-del-1s")
processor = Wav2Vec2Processor.from_pretrained("42MARU/ko-spelling-wav2vec2-conformer-del-1s")

Downloading:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/720M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:752: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

sr은 sample late로 여기서 조절하여 결과값 조절할수있다.

In [ ]:
audio, sampling_rate = librosa.load("/content/성수동2가-3-1.wav",sr=17500)
audio,sampling_rate

(array([0.        , 0.        , 0.        , ..., 0.01371877, 0.01301922,
        0.01327659], dtype=float32), 17500)

In [ ]:
display.Audio("/content/성수동2가-3-1.wav", autoplay=False)

In [ ]:
answer = '2022년 카타르 월드컵 조별리그 1차전 우루과이전이 이제 이틀 앞으로 다가왔다.'

In [ ]:
input_values = tokenizer(audio, return_tensors = 'pt').input_values
input_values

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0137, 0.0130, 0.0133]])

In [ ]:
logits = model(input_values).logits

In [ ]:
predicted_ids = torch.argmax(logits, dim =-1)
transcriptions = tokenizer.decode(predicted_ids[0])

In [ ]:
transcriptions

'어 2022년 카타리 월드컵 조별 리그 1차 전 우류과의전이 이제 이틀 앞으로 다가왔다'

In [ ]:
answer

'2022년 카타르 월드컵 조별리그 1차전 우루과이전이 이제 이틀 앞으로 다가왔다.'

In [ ]:
print(transcriptions)
print(answer)

어 2022년 카타리 월드컵 조별 리그 1차 전 우류과의전이 이제 이틀 앞으로 다가왔다
2022년 카타르 월드컵 조별리그 1차전 우루과이전이 이제 이틀 앞으로 다가왔다.
